## Notebook used to recommand a packaging material. 

In [1]:
import pandas as pd
import numpy as np
import random
import time
import math
import geopy.distance

In [2]:
import warnings
warnings.filterwarnings("ignore")

Dataset upload

In [4]:
materiaux = pd.read_csv("data/emballages.csv")

In [5]:
materiaux.head()

,Catégorie,Matière,kg équivalent CO2 / tonne,Taux de recyclage,Id matière,Recyclé,Type emballage
0,Papier,Pâte à papier primaire,297.0,62.0,1,0,Pochette
1,Papier,Pâte à papier primaire recyclée,317.0,52.0,2,1,Pochette
2,Papier,Pâte à papier Eucalyptus,440.0,61.5,3,0,Pochette
3,Papier,Pâte à papier Résineux,410.0,56.5,4,0,Pochette
4,Papier,Pâte à papier Feuillus,400.0,60.5,5,0,Pochette


In [6]:
materiaux.columns[6]

'Type emballage'

Changer unité de la mesure de carbon en kg CO2 / kg

In [7]:
materiaux['kg équivalent CO2 / tonne'] = materiaux['kg équivalent CO2 / tonne']/1000
materiaux.columns = ['Catégorie', 'Matière', 'kg CO2 / kg', 'Taux de recyclage', 'Id matière', 'Recyclé', 'Type emballage']

In [8]:
materiaux

,Catégorie,Matière,kg CO2 / kg,Taux de recyclage,Id matière,Recyclé,Type emballage
0,Papier,Pâte à papier primaire,0.297,62.0,1,0,Pochette
1,Papier,Pâte à papier primaire recyclée,0.317,52.0,2,1,Pochette
2,Papier,Pâte à papier Eucalyptus,0.440,61.5,3,0,Pochette
3,Papier,Pâte à papier Résineux,0.410,56.5,4,0,Pochette
4,Papier,Pâte à papier Feuillus,0.400,60.5,5,0,Pochette
...,...,...,...,...,...,...,...
149,Bois,Bois primaire,0.964,61.0,39,0,Boîte
150,Bois,Bois primaire,0.964,61.0,39,0,Palette
151,Bois,Bois primaire,0.964,61.0,39,0,Fourchette
152,Bois,Bois primaire,0.964,61.0,39,0,Couteau


In [9]:
df = materiaux
mat = ['Plastique transparent', 'Bouteille ']
x_0 = df[df['Matière'] == mat[0]][df[df.columns[6]] == mat[1]]

In [10]:
x_0

,Catégorie,Matière,kg CO2 / kg,Taux de recyclage,Id matière,Recyclé,Type emballage
86,Plastique,Plastique transparent,2.35,61.0,12,0,Bouteille


Créer un score comparatif entre 0.5 et 1.5 par rapport aux emballages initiaux 

In [29]:
def recommandation(df, emballage_init, wc=0.7, wrr=0.2, wr=0.1):
    n = len(emballage_init)
    
    reco = [0,0]
    i = 0
    
    # normaliser les émissions carbone
    df['score carbone'] = df['kg CO2 / kg'].apply(lambda x: (x- min(df['kg CO2 / kg']))/(max(df['kg CO2 / kg'])-min(df['kg CO2 / kg'])))       

    for mat in emballage_init: 
        #définir le materiau initial qu'on veut remplacer
        x_0 = df[df['Matière'] == mat[0]][df[df.columns[6]] == mat[1]].reset_index(drop=True)
        #choisir fermes qui produisent le meme produit uniquement
        sub_df = df[df[df.columns[6]] == mat[1]]                                                                                                                  
        
        # appliquer la formule du score packaging
        sub_df['score packaging'] = wc - ((sub_df['score carbone']) - wrr * sub_df['Taux de recyclage']/100 - wr * sub_df['Recyclé'])
    
        sub_df = sub_df.sort_values('score packaging', ascending = False)
        reco[i] = sub_df
        i = i+1
        
    
    return reco

In [23]:
materiaux.columns

Index(['Catégorie', 'Matière', 'kg CO2 / kg', 'Taux de recyclage',
       'Id matière', 'Recyclé', 'Type emballage'],
      dtype='object')

Définir emballage initial

In [30]:
emballage_init = [['Plastique transparent', 'Bouteille '], ['Pâte à papier Eucalyptus', 'Étiquette']]

In [31]:
reco = recommandation(materiaux, emballage_init)

In [32]:
reco[0].iloc[:10,:]

,Catégorie,Matière,kg CO2 / kg,Taux de recyclage,Id matière,Recyclé,Type emballage,score carbone,score packaging
89,Plastique,Plastique 100% ex-recyclé,0.202,36.0,13,1,Bouteille,0.000000,0.872000
87,Plastique,Plastique blanc,2.025,63.0,11,0,Bouteille,0.242469,0.583531
88,Plastique,Plastique,2.150,59.0,12,0,Bouteille,0.259094,0.558906
86,Plastique,Plastique transparent,2.350,61.0,12,0,Bouteille,0.285695,0.536305


In [33]:
reco[1].iloc[:10,:]

,Catégorie,Matière,kg CO2 / kg,Taux de recyclage,Id matière,Recyclé,Type emballage,score carbone,score packaging
15,Papier,Pâte à papier primaire recyclée,0.317,52.0,2,1,Étiquette,0.015296,0.888704
19,Papier,Pâte à papier Papier recyclé,0.470,54.0,6,1,Étiquette,0.035645,0.872355
14,Papier,Pâte à papier primaire,0.297,62.0,1,0,Étiquette,0.012635,0.811365
18,Papier,Pâte à papier Feuillus,0.400,60.5,5,0,Étiquette,0.026335,0.794665
16,Papier,Pâte à papier Eucalyptus,0.440,61.5,3,0,Étiquette,0.031655,0.791345
17,Papier,Pâte à papier Résineux,0.410,56.5,4,0,Étiquette,0.027665,0.785335
20,Papier,Papier moyen,0.919,61.0,7,0,Étiquette,0.095365,0.726635
